# Deterministic guarantees with Output Range Analysis
In this tutorial we will show how to guarantee feasibility and even stability for AMPC on linear systems using Output Range Analysis. First, we will define our system and train an AMPC using do-mpc.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import sys
from casadi import *
from do_mpc.tools import Timer
import jdc
# Add do_mpc to path. This is not necessary if it was installed via pip
import os
rel_do_mpc_path = os.path.join('..','..','..')
sys.path.append(rel_do_mpc_path)

# Import do_mpc package:
import do_mpc

## Model
We consider the oscillating mass example as described in the original paper and in our do-it-yourself AMPC notebook. That is the model is defined as a discrete 2D-model


In [ ]:
model_type = 'discrete' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)
x=model.set_variable('_x','x',(2,1))
u=model.set_variable('_u','u')
A=np.array([[0.5403, -0.8415],[0.8415, 0.5403]])
B=np.array([[-0.4597],[0.8415]])
model.set_rhs('x',A@x+B@u)
model.setup()

## Calculate LQR gain, MPI set for stability
Now there is one difference. We are now looking to stabilize our MPC and later our AMPC. For this, we need to get the LQR feedback controller and the control invariant set in which we ensure to remain, if the LQR is used. Additionally, we need the terminal cost. The following algorithm iteratively computes this maximum control invariant set

In [ ]:

from control import dare
from pypoman import compute_polytope_vertices
from pypoman.polygon import plot_polygon
from numpy.linalg import matrix_power as mp

# state constraints matrix
F = np.array([[1 / 5, 0],
              [-1 / 5, 0],
              [0, 1 / 5],
              [0, -1 / 5]])

# input constraints matrix
G = np.array([[1],
              [-1]])

Q=np.array([[2,0],[0,2]])

R=np.array([1])

# Compute optimal feedback matrix by solving the DARE
P_lqr, _, K_lqr = dare(A, B, Q, R)
print(K_lqr)

# convert to numpy array
K_lqr = -np.array(K_lqr)

# Update the closed-loop
A_cl = A + B @ K_lqr
# Maximum number of iterations
max_iter = 100
print(A_cl)

# Initialize matrix describing the invariant set
invariant_mat = np.zeros((0, 2))

# run loop
for n_f in range(max_iter):

    # extend the matrix describing the invariant set
    invariant_mat = np.vstack([invariant_mat, F @ mp(A_cl, n_f), G @ K_lqr @ mp(A_cl, n_f)])

    # termination criterion
    one_vec = np.ones((invariant_mat.shape[0], 1))

    # compute vertices of current iterate of the maximum invariant set
    verts = compute_polytope_vertices(invariant_mat, one_vec)

    # compute predecessor states of the current vertices
    verts_next = [A_cl @ np.reshape(vert, (-1, 1)) for vert in verts]

    # check if all verts lie inside the current iterate of the maximum invariant set
    in_omega = [all(invariant_mat @ vert <= one_vec) for vert in verts_next]

    # if all predecessor verts inside the current iterate of the maximum invariant set -> break
    if all(in_omega):
        print('Algorithm converged after ' + str(n_f + 1) + ' step(s).')
        break
plot_polygon(verts,color='r',alpha=0.4,resize=True)
#print(invariant_mat)

## MPC
Now we can use this MPI set as the terminal set, the cost of the LQR as the terminal cost in our MPC. This adapts the MPC to the following code in do-mpc

In [35]:
mpc = do_mpc.controller.MPC(model)
setup_mpc = {
    'n_horizon': 3,
    'n_robust': 0,
    'open_loop': 0,
    't_step': 0.1,
}
mpc.set_param(**setup_mpc)
mpc.settings.supress_ipopt_output()
_x=model.x['x']
_u=model.u['u']
lterm=_x.T@Q@_x+_u.T@R@_u
mterm=_x.T@P_lqr@_x
mpc.set_objective(mterm,lterm)

mpc.bounds['lower','_x','x']=np.array([[-5],[-5]])
mpc.bounds['upper','_x','x']=np.array([[5],[5]])
mpc.bounds['lower','_u','u']=np.array([[-1]])
mpc.bounds['upper','_u','u']=np.array([[1]])
mpc.prepare_nlp()
#print(mpc.opt_x['_x'])
extra_cons= invariant_mat@mpc.opt_x['_x', -1, 0][0][0:2]-1
mpc.nlp_cons.append(
       extra_cons
    )
mtx=np.zeros(extra_cons.shape)
mtx.fill(-inf)

mpc.nlp_cons_lb.append(mtx)
mpc.nlp_cons_ub.append(np.zeros(extra_cons.shape))
mpc.create_nlp()

C:\Users\smjsadam\Documents\Job\Forschung\Code\2025-workshop-dycops\do_mpc\controller\_mpc.py:885: UserWarning: rterm was not set and defaults to zero. Changes in the control inputs are not penalized. Can lead to oscillatory behavior.
  warnings.warn('rterm was not set and defaults to zero. Changes in the control inputs are not penalized. Can lead to oscillatory behavior.')


## Estimator and Simulator
Estimator and Simulator remain the same.

In [ ]:
estimator = do_mpc.estimator.StateFeedback(model)
simulator=do_mpc.simulator.Simulator(model)
simulator.set_param(t_step = 0.1)
simulator.setup()

## Visualization and closed-loop simulation
Of course, we can again visualize our stable MPC.

In [ ]:
np.random.seed(99)

# Initial state
mpc.reset_history()
simulator.reset_history()
e = np.ones([model.n_x,1])
x0 = np.random.uniform(-3*e,3*e) # Values between +3 and +3 for all states
mpc.x0 = x0
print(x0)
simulator.x0 = x0
estimator.x0 = x0

# Use initial state to set the initial guess.
mpc.set_initial_guess()
for k in range(10):
    u0 = mpc.make_step(x0)
    y_next = simulator.make_step(u0)
    x0 = estimator.make_step(y_next)
from matplotlib import rcParams
rcParams['axes.grid'] = True
rcParams['font.size'] = 18
fig, ax, graphics = do_mpc.graphics.default_plot(mpc.data, figsize=(16,9))
graphics.plot_results()
graphics.reset_axes()
plt.show()

## Approximate MPC
Now lets define our approximate MPC as before in the first notebook. We see how efficiently we can generate the AMPC using do-mpc

In [ ]:
approx_mpc=do_mpc.approximateMPC.ApproxMPC(mpc)
approx_mpc.settings.act_fn='relu'
n_l=10
L=1
approx_mpc.settings.n_hidden_layers=L
approx_mpc.settings.n_neurons=n_l
approx_mpc.settings.scaling=False
approx_mpc.setup()
n_samples=2000
sampler=do_mpc.approximateMPC.Sampler(mpc)
sampler.settings.n_samples=n_samples
sampler.setup()
#sampler.default_sampling()
trainer=do_mpc.approximateMPC.Trainer(approx_mpc)
trainer.settings.n_samples=n_samples
trainer.settings.n_epochs=3000
trainer.setup()
trainer.default_training()


## Closed_loop simulation
Visualization and simulation are identical to the MPC class, we only have to use the simulator data.

In [ ]:
np.random.seed(99)
# Initial state
mpc.reset_history()
simulator.reset_history()
e = np.ones([model.n_x,1])
x0 = np.random.uniform(-3*e,3*e) # Values between +3 and +3 for all states
mpc.x0 = x0
simulator.x0 = x0
estimator.x0 = x0

# Use initial state to set the initial guess.
mpc.set_initial_guess()
for k in range(10):
    u0 = approx_mpc.make_step(x0,clip_to_bounds=False)
    y_next = simulator.make_step(u0)
    x0 = estimator.make_step(y_next)
from matplotlib import rcParams
rcParams['axes.grid'] = True
rcParams['font.size'] = 18
fig, ax, graphics = do_mpc.graphics.default_plot(simulator.data, figsize=(16,9))
graphics.plot_results()
graphics.reset_axes()
plt.show()

## Calculate X_in
What we require is a set of X_in in which we guarantee to safely use the AMPC


In [1]:
import polytope
import pickle as pkl
from pathlib import Path
import torch
data_dir=trainer.settings.data_dir
data_dir = Path(data_dir)
data_dir = data_dir.joinpath("data_n" + str(n_samples) + "_opt.pkl")

with open(data_dir, "rb") as f:
    dataset = pkl.load(f)
x0 = torch.tensor(dataset["x0"], dtype=trainer.approx_mpc.torch_data_type).reshape(
            -1, trainer.approx_mpc.mpc.model.n_x
        )
x0=np.array(x0.detach().cpu())
plt.scatter(x0[:,0],x0[:,1])
N_pred=3
C_in=invariant_mat
c_in=one_vec
plot_polygon(verts,color='r',alpha=0.4,resize=True)
for k in range(N_pred):
    X_mat=np.zeros((len(F)+len(G)+len(C_in),3))
    X_mat[0:F.shape[0],0:F.shape[1]]=F
    X_mat[F.shape[0]:F.shape[0]+G.shape[0],F.shape[1]:F.shape[1]+G.shape[1]]=G
    #X_mat[6:8,3:4]=G
    #X_mat[8:10,4:5]=G
    #X_mat[10:14,0:3]=np.concatenate((F@A,F@B),axis=1)
    #X_mat[14:18,0:4]=np.concatenate((F@A**2,F@A@B,F@B),axis=1)
    X_mat[F.shape[0]+G.shape[0]:F.shape[0]+G.shape[0]+len(C_in),0:F.shape[1]+G.shape[1]]=np.concatenate((C_in@A,C_in@B),axis=1)#,invariant_mat@A**2@B,invariant_mat@A@B
    ones=np.concatenate((np.ones((F.shape[0]+G.shape[0],1)),c_in),axis=0)
    #plot_polygon(verts,color='r',alpha=0.4,resize=True)

    verts_new=compute_polytope_vertices(X_mat,ones)
    verts_projected=[verts_new[k][0:model.n_x] for k in range(len(verts_new))]
    #print(verts_projected)
    poly=polytope.qhull(np.array(verts_projected))
    C_in=poly.A
    c_in=poly.b.reshape((len(poly.b),1))
    #print(c_in)
plot_polygon(verts_projected,resize=True)

NameError: name 'trainer' is not defined

## Output Range analysis for U_1 and forward propagation for X_1
Next, we prepare the first optimization problem to certify the output range analysis

In [2]:
import cvxpy as cp
M=1e5

u_res=np.zeros((len(G),1))
for i in range(len(G)):
    t=cp.Variable(n_l, integer=True)
    z=cp.Variable(n_l)
    u0=cp.Variable(model.n_u)
    #print(G[i]*u0)
    x0=cp.Variable(model.n_x)
    W_1=approx_mpc.net.state_dict()['layers.0.weight']
    b_1=approx_mpc.net.state_dict()['layers.0.bias']
    W_2=approx_mpc.net.state_dict()['layers.2.weight']
    b_2=approx_mpc.net.state_dict()['layers.2.bias']

    objective = cp.Maximize(G[i]@u0)
    prob = cp.Problem(objective,[x0@W_1.T+b_1 <= z,0<=z, x0@W_1.T+b_1+M*t>=z,M*(np.ones(n_l)-t)>=z,t<=1,t>=0,C_in@x0<=c_in,u0==z@W_2.T+b_2])
    prob.solve()
    u_res[i]=u0.value
    print(x0.value)
c_u=G*u_res
print(c_u)



NameError: name 'np' is not defined

In [ ]:

x=np.array([[-1.63464],[ -2.72462]])
print(mpc.make_step(x))
print(approx_mpc.make_step(x,clip_to_bounds=False))

## Output Range analysis for k=1
What we require is a second optimization problem that does a set range analysis for k steps

In [ ]:

def evaluate_set(C_in,c_in,k,C_out):
    W_1=approx_mpc.net.state_dict()['layers.0.weight']
    b_1=approx_mpc.net.state_dict()['layers.0.bias']
    W_2=approx_mpc.net.state_dict()['layers.2.weight']
    b_2=approx_mpc.net.state_dict()['layers.2.bias']
    M=1e5
    xki_res=np.zeros((len(C_out),model.n_x))
    for i in range(len(C_out)):
        constraints=[]
        t=cp.Variable((k,n_l), integer=True)
        z0=cp.Variable((k,model.n_x))
        z1=cp.Variable((k,n_l))
        u=cp.Variable((k,model.n_u))
        x0=cp.Variable(model.n_x)
        xki=cp.Variable(model.n_x)
        constraints.append(C_in@x0<=c_in)
        constraints.append(A@z0[k-1]+B@u[k-1]==xki)
        constraints.append(z0[0]==x0)
        for j in range(k):
            if j>0:
                constraints.append(z0[j]==A@z0[j-1]+B@u[j-1])
            constraints.append(z0[j]@W_1.T+b_1 <= z1[j])
            constraints.append(z1[j]>=0)
            constraints.append(z0[j]@W_1.T+b_1 + M*t[j] >= z1[j])
            constraints.append(M*(np.ones(n_l)-t[j])>=z1[j])
            constraints.append(t[j]<=1)
            constraints.append(t[j]>=0)
            constraints.append(u[j]==z1[j]@W_2.T+b_2)
        objective = cp.Maximize(C_out[i]@xki)
        prob = cp.Problem(objective,constraints)
        prob.solve(verbose=False)
        xki_res[i]=xki.value
        #print(xki.value)
    return xki_res
xki_res=evaluate_set(C_in,c_in,1,C_in)
#print(xki_res)
c_x=np.zeros((len(C_in),1))
for m in range(len(C_in)):
    c_x[m]=C_in[m,:]@xki_res[m,:].reshape((model.n_x,1))

vertices_x=compute_polytope_vertices(C_in,c_x)
vertices_inv=compute_polytope_vertices(C_in,c_in)
plot_polygon(vertices_x,resize=True)
plot_polygon(vertices_inv,resize=True,color='r')

## Stability with LQR adaption of neural network
Now we have a feasible controller. For stability the controller needs to bring the system in a terminal set after some k steps. Secondly, we also need to ensure that within this terminal set we will reach the origin. While the first part can be easily verified with the same optimization problem we used for the recursive feasibility, the second part is not in general true for neural networks.
At first, we need the intersection of the terminal set with the region around the origin where there is no change in the ReLU activation functions of the neural network. Lets calculate this region

In [ ]:
from itertools import combinations
x1=np.linspace(-10,10,200)
fig,ax=plt.subplots()
for k in range(len(W_1)):
    W_1_k_1=W_1[k,0]
    W_1_k_2=W_1[k,1]
    b_1_k=b_1[k]
    x2=(b_1_k-W_1_k_1*x1)/W_1_k_2
    plt.plot(x1,x2)
z_1=torch.Tensor([0,0])@W_1.T+b_1
gamma=np.array((b_1<0).detach()).reshape(-1,1)

print(np.array(b_1.detach().cpu()))
lines=list(np.concatenate((np.array(W_1.detach().cpu()),np.array(b_1.detach().cpu()).reshape((-1,1))),axis=1))
vertices_list=[]
for l1, l2 in combinations(lines, 2):
    A = np.array([[l1[0], l1[1]], [l2[0], l2[1]]])
    b = np.array([l1[2], l2[2]])

    if np.linalg.det(A) != 0:
        x = np.linalg.solve(A, b)
        if x[0]<4 and x[0]>-6 and x[1]<3 and x[1]>-3:
            vertices_list.append(x)
            plt.plot(x[0], x[1], 'ro')
            print(x[1])
        else:
            plt.plot(x[0], x[1], 'bo')
        #plt.plot(x[0], x[1], 'ro')
ax.set_xlim(-7,5)
ax.set_ylim(-5,5)

Now we can compute our new terminal set as the intersection

In [ ]:
plot_polygon(vertices_list,resize=True)
poly=polytope.qhull(np.array(vertices_list))
R_as=poly.A
r_as=poly.b.reshape((len(poly.b),1))

terminal_set_mat=np.concatenate((invariant_mat,R_as),axis=0)
terminal_set_vec=np.concatenate((one_vec,r_as),axis=0)
terminal_set=compute_polytope_vertices(terminal_set_mat,terminal_set_vec)
plot_polygon(terminal_set,resize=True,color='r')

Now we have to adjust the values of the last layer weights such that we can guarentee the stability

In [ ]:

W_hat=cp.Variable((model.n_u,n_l))
b_hat=cp.Variable((model.n_u,1))
objective = cp.Minimize(cp.sum_squares(np.array(W_2.cpu().detach())-W_hat)+cp.sum_squares(np.array(b_2.cpu().detach())-b_hat))
constraints=[]
W_eq=gamma*np.array(W_1.cpu().detach())
b_eq=gamma.T*np.array(b_1.cpu().detach())
constraints.append(W_hat@W_eq==-K_lqr)
constraints.append(W_hat@b_eq.reshape((-1,1))+b_hat==0)
prob = cp.Problem(objective,constraints)
prob.solve()
print(W_hat.value)
print(W_2)
print(b_2)
print(b_hat.value)
approx_mpc.net.state_dict()['layers.2.weight']=torch.Tensor(W_hat.value)
approx_mpc.net.state_dict()['layers.2.bias']=torch.Tensor(b_hat.value)


Let's take a lot at the closed-loop response of this new adapted AMPC.

In [ ]:
np.random.seed(99)
# Initial state
mpc.reset_history()
simulator.reset_history()
e = np.ones([model.n_x,1])
x0 = np.random.uniform(-3*e,3*e) # Values between +3 and +3 for all states
mpc.x0 = x0
simulator.x0 = x0
estimator.x0 = x0

# Use initial state to set the initial guess.
mpc.set_initial_guess()
for k in range(10):
    u0 = approx_mpc.make_step(x0,clip_to_bounds=False)
    y_next = simulator.make_step(u0)
    x0 = estimator.make_step(y_next)
from matplotlib import rcParams
rcParams['axes.grid'] = True
rcParams['font.size'] = 18
fig, ax, graphics = do_mpc.graphics.default_plot(simulator.data, figsize=(16,9))
graphics.plot_results()
graphics.reset_axes()
plt.show()

## Output Range analysis for k=6 for guaranteeing stability
Finally, we can use the same optimization problem as before to guarantee that we will remain in the terminal set

In [ ]:
xki_res=evaluate_set(C_in,c_in,2,terminal_set_mat)
#print(xki_res)
c_x=np.zeros((len(terminal_set_mat),1))
for m in range(len(terminal_set_mat)):
    c_x[m]=terminal_set_mat[m,:]@xki_res[m,:].reshape((model.n_x,1))

#print(c_x)
vertices_x=compute_polytope_vertices(terminal_set_mat,c_x)
vertices_inv=compute_polytope_vertices(terminal_set_mat,terminal_set_vec)
plot_polygon(vertices_x,resize=True)
plot_polygon(vertices_inv,resize=True,color='r')